In [ ]:
# load ContGridMod package
using Pkg
Pkg.activate("..")
using ContGridMod
using Plots

In [ ]:
# create the mesh
dx = 0.011;
mesh = get_lattice_mesh("../data/borders/euro_border.json", dx)

# load the discrete model
dm = load_discrete_model("../data/ml/training_2.h5", mesh.scale_factor)

# distribute parameters
contmod = get_params(mesh, "../data/ml/training_2.h5", Niter=10,
    dmax = 2*dx, patch=100.0, tau=5.0e-6);

# update the model with values from the discrete model
update_model!(contmod, "../data/ml/susceptances.h5")

# compute the stable solution
compute_stable_sol!(contmod);

In [ ]:
# add a 900MW loss at the location of the first generator
# add_local_disturbance!(contmod, dm.coord[1,:], -9.0, 0.02);

# or alternatively at a given coordinate (p.s. don't uncomment both or you will get two faults)
add_local_disturbance!(contmod, dm.coord[1,:], -9.0, 0.02);
# gps_coord = [39.214297918336996, -1.0546984597003388]
# ContGridMod.add_local_disturbance_with_gps!(contmod, gps_coord, -9.0, 0.02);

In [ ]:
# integrate the system dynamics
ts, thetas, omegas = perform_dyn_sim(contmod, interval = 1,
    Ndt = 2500, dt = 0.01, method = "crank-nicolson");

In [ ]:
# print the theoretical post-fault frequency
omega_pf = sum(contmod.dp) / sum(contmod.d)
println("post-fault frequency: $omega_pf")

In [ ]:
# plot the evolution the frequency at different locations
# To add a gps coordinate:
# 1) go to maps.google.com
# 2) left-click at the selected location (do it multiple times if nothing happens or try to zoom out)
# 3) right-click on token that appeared, then click on lat-lon coords to copy them

plot_coord = [37.51944333869843 14.103063062652804;
47.614422136153 6.456579347156401;
39.631982868867446 -4.963330785436443;
44.76193867093294 23.361757829639277]

time_plot(contmod, ts, omegas, plot_coord, borders=[mesh.border])

In [ ]:
ts, omegas = disc_dynamics(dm, 0.0, 25.0, -9.0; faultid=1, dt=1e-2);

In [ ]:
plot_coord_proj = albers_projection(plot_coord ./ (180 / pi) );
plot_coord_proj ./= contmod.mesh.scale_factor;
disc_ids = ContGridMod.get_discrete_id(dm.coord[dm.id_gen[dm.p_gen .> 0], :], plot_coord_proj);

In [ ]:
plot(ts, omegas[:, dm.id_gen[disc_ids]], legend=false)